In [1]:
# Libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

---

<h1><center>Data Pre-Processing

---

In [2]:
# Download data
df = pd.read_csv('ev_battery_charging_data.csv')

In [3]:
# Drop rows with any NaN values
df_clean = df.dropna()

# Shape after cleaning
print("\nAfter removing rows with NaN:")
print(f"Rows: {df_clean.shape[0]}, Columns: {df_clean.shape[1]}")


After removing rows with NaN:
Rows: 1000, Columns: 13


In [4]:
# Drop inputs calculated from the column we are predicting
drop_cols = ['Degradation Rate (%)', 'Efficiency (%)', 'Optimal Charging Duration Class']
df_clean = df_clean.drop(columns=drop_cols)

In [5]:
# List the categorical columns and numerical columns for one-hot encoding later
cat_col = ['Charging Mode', 'Battery Type', 'EV Model']
num_col = ['SOC (%)', 'Voltage (V)', 'Current (A)', 'Battery Temp (°C)', 'Ambient Temp (°C)', 'Charging Cycles']
target = 'Charging Duration (min)'

In [6]:
# Move the column we are predicting to the end
df_clean[target] = df_clean.pop(target)

In [7]:
# Split the data into test and train
from sklearn.model_selection import train_test_split
X = df_clean.drop(columns=target)
y = df_clean[target]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

In [8]:
df_clean.head()

,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Mode,Battery Type,Charging Cycles,EV Model,Charging Duration (min)
0,43.708611,3.629593,33.553512,33.454060,26.439918,Fast,Li-ion,112,Model B,59.363552
1,95.564288,3.879331,32.228092,35.933628,31.108647,Fast,LiFePO4,398,Model A,67.343566
2,75.879455,4.111062,91.562912,25.009358,30.203219,Slow,LiFePO4,175,Model A,105.454739
3,63.879264,4.012557,32.459158,32.497482,18.077998,Fast,LiFePO4,150,Model B,54.000439
4,24.041678,4.064593,34.475475,31.434920,17.984989,Fast,Li-ion,886,Model C,106.964968


In [9]:
df_clean.describe(include='all')

,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Mode,Battery Type,Charging Cycles,EV Model,Charging Duration (min)
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000,1000.000000,1000,1000.000000
unique,NaN,NaN,NaN,NaN,NaN,3,2,NaN,3,NaN
top,NaN,NaN,NaN,NaN,NaN,Slow,LiFePO4,NaN,Model C,NaN
freq,NaN,NaN,NaN,NaN,NaN,341,503,NaN,343,NaN
mean,54.123090,3.854912,55.216515,29.807501,24.882106,NaN,NaN,556.560000,NaN,69.846449
std,26.292363,0.204533,26.160678,5.729787,5.736199,NaN,NaN,263.763052,NaN,28.919481
min,10.416882,3.502253,10.001047,20.013068,15.000614,NaN,NaN,101.000000,NaN,20.618382
25%,31.237594,3.668752,33.521588,24.838015,19.899480,NaN,NaN,317.750000,NaN,44.938000
50%,54.712664,3.863114,55.055253,29.685586,24.891962,NaN,NaN,571.000000,NaN,69.040282
75%,76.988763,4.032326,78.319317,34.750816,29.799869,NaN,NaN,786.000000,NaN,93.985178


In [10]:
# df_clean[target].plot(title='Charge Duration', xlabel='Index', ylabel='Time (min)')


---

<h1><center>Linear Regression Models

---

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [12]:
# Preprocessor for numerical columns
num_transformer = Pipeline(
    steps=[("scaler", StandardScaler())])

# Preprocessor for categorical columns
cat_transformer = Pipeline(
    steps=[("encoder", OneHotEncoder(drop='first'))])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_col),
        ('cat', cat_transformer, cat_col)])

In [13]:
# Linear regression with feature selection
linreg_pipe = Pipeline([('preprocessor', preprocessor), 
                     ('linreg', LinearRegression())])

# Fit the model
linreg_pipe.fit(Xtrain, ytrain)

,steps,"[('preprocessor', ...), ('linreg', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:
# Lasso regression
alphas = np.logspace(-4, 4, 10)
lasso_models = []

for alpha in alphas:
    lasso_pipe = Pipeline([('preprocessor', preprocessor), 
                         ('lasso', Lasso(alpha=alpha))])
    lasso_pipe.fit(Xtrain, ytrain)
    lasso_models.append(lasso_pipe)

In [15]:
pred = linreg_pipe.predict(Xtest)
r2 = r2_score(ytest, pred)
print(r2)

-0.02910941045986326


In [16]:
linreg_mae = mean_absolute_error(ytest, pred)
base_mae = mean_absolute_error(ytest, np.full_like(ytest, ytrain.mean()))
print(linreg_mae)
print(base_mae)



24.86825124211276
24.601615714754136


In [17]:
pred = lasso_models[9].predict(Xtest)
r2 = r2_score(ytest, pred)
print(r2)

-0.000998072902663738


<h2><center>Logistic Regression Comparison

In [18]:
from sklearn.linear_model import LogisticRegression

---

<h1><center>Neural Networks

---

Plan:
1. Build a basic NN to get results
    a. Build pipeline
    b. Build NN model
    c. Compile NN w/ pipeline
    d. Calculate R2 score and compare with linreg
2. Build function to build pipelines 

In [19]:
# Split the data into train, validate, test

XtrainNN, XvalidNN, ytrainNN, yvalidNN = train_test_split(Xtrain, ytrain, test_size=0.2)

In [20]:
# Preprocessor for numerical columns
num_transformer = Pipeline(
    steps=[("scaler", StandardScaler())])

# Preprocessor for categorical columns
cat_transformer = Pipeline(
    steps=[("encoder", OneHotEncoder(drop='first'))])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_col),
        ('cat', cat_transformer, cat_col)])

In [21]:
XtrainNN


,SOC (%),Voltage (V),Current (A),Battery Temp (°C),Ambient Temp (°C),Charging Mode,Battery Type,Charging Cycles,EV Model
715,93.657623,4.196406,36.982047,26.237589,21.417091,Slow,LiFePO4,947,Model A
131,29.989703,3.847113,11.134677,36.459676,19.357828,Normal,Li-ion,455,Model A
734,42.294744,3.591497,94.845189,24.933172,23.610716,Fast,LiFePO4,612,Model B
934,25.451136,3.973528,73.838875,26.144328,24.780968,Slow,Li-ion,176,Model C
58,14.070456,3.835913,68.587612,21.511672,32.798268,Normal,LiFePO4,713,Model C
...,...,...,...,...,...,...,...,...,...
667,24.461324,4.151417,68.647578,23.213592,31.777077,Normal,Li-ion,406,Model C
688,79.254364,3.924377,54.357813,21.323304,25.885147,Normal,Li-ion,130,Model A
860,79.797486,4.043513,88.009332,36.293420,27.508669,Slow,LiFePO4,990,Model B
372,18.499866,4.098070,72.324354,25.292701,16.023004,Slow,Li-ion,715,Model B


In [22]:
XtrainNN_norm = preprocessor.fit_transform(XtrainNN)
XvalidNN_norm = preprocessor.transform(XvalidNN)
Xtest_norm = preprocessor.transform(Xtest)
feature_names_x = preprocessor.get_feature_names_out()
XtrainNN_norm_df = pd.DataFrame(XtrainNN_norm, columns=feature_names_x, index=XtrainNN.index)
XvalidNN_norm_df = pd.DataFrame(XvalidNN_norm, columns=feature_names_x, index=XvalidNN.index)
Xtest_norm_df = pd.DataFrame(Xtest_norm, columns=feature_names_x, index=Xtest.index)
Xtrain_full_norm = preprocessor.fit_transform(Xtrain)
Xtrain_full_norm_df = pd.DataFrame(Xtrain_full_norm, columns=feature_names_x, index=Xtrain.index)

y_scaler = StandardScaler()
ytrainNN_norm = y_scaler.fit_transform(ytrainNN.values.reshape(-1,1))
yvalidNN_norm = y_scaler.transform(yvalidNN.values.reshape(-1,1))
ytest_norm = y_scaler.transform(ytest.values.reshape(-1,1))
feature_names_y = [ytrainNN.name]
ytrainNN_norm_df = pd.DataFrame(ytrainNN_norm, columns=feature_names_y, index=ytrainNN.index)
yvalidNN_norm_df = pd.DataFrame(yvalidNN_norm, columns=feature_names_y, index=yvalidNN.index)
ytest_norm_df = pd.DataFrame(ytest_norm, columns=feature_names_y, index=ytest.index)
ytrain_full_norm = y_scaler.fit_transform(ytrain.values.reshape(-1,1))
ytrain_full_norm_df = pd.DataFrame(ytrain_full_norm, columns=feature_names_y, index=ytrain.index)

In [24]:
# Build an MLP

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import GlorotUniform

ki = GlorotUniform(seed=2434)

model_mlp = Sequential([
    Dense(64, input_shape=(XtrainNN_norm_df.shape[1],), activation='relu', kernel_initializer=ki),
    Dense(32, activation='relu', kernel_initializer=ki),
    Dense(16, activation='relu', kernel_initializer=ki),
    Dense(1, kernel_initializer=ki)
])

model_mlp.compile( optimizer="rmsprop", loss="mse", metrics=["mae"] )

/opt/miniconda3/envs/ME276/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# Train the MLP

history_mlp = model_mlp.fit(x=XtrainNN_norm_df, y=ytrainNN_norm_df, epochs=10, validation_data=(XvalidNN_norm_df,yvalidNN_norm_df))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0391 - mae: 0.8812 - val_loss: 0.8673 - val_mae: 0.7726
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9717 - mae: 0.8560 - val_loss: 0.8637 - val_mae: 0.7720
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9495 - mae: 0.8458 - val_loss: 0.8613 - val_mae: 0.7752
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9308 - mae: 0.8342 - val_loss: 0.8731 - val_mae: 0.7767
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9091 - mae: 0.8265 - val_loss: 0.8925 - val_mae: 0.7871
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8955 - mae: 0.8210 - val_loss: 0.8874 - val_mae: 0.7882
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8798 - mae: 0.8124 - val_loss: 0.9054 - val_mae: 0.7943
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8677 - mae: 0.8050 - val_loss: 0.9407 - val_mae: 0.8042
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8584 - mae: 

In [26]:
mlp_perf = r2_score(ytrainNN_norm_df, model_mlp.predict(XtrainNN_norm_df))
mlp_perf_valid = r2_score(yvalidNN_norm_df, model_mlp.predict(XvalidNN_norm_df))
print(mlp_perf)
print(mlp_perf_valid)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
0.1887432336807251
-0.13774514198303223


In [27]:
# Next Steps:
# 1. Confirm R2

# A: Yes, it takes the average of the training data, then in the denom does each training point minus the mean, and in the num does the trianing point minus the prediction, and divides to compare.
#     Performs better with the training data because the model has seen this data and has learned the patters of it better. However, the model has not seen the validation data. The positive training
#     R2 and negative validation R2 is a sign of overfitting. Could compare MAE for further investigation. 

# 2. NN Settings and Why

# A: Vary the following:
    # - Number of layers 1 --> 5
    # - Units per layer (16, 32, 64, 128, 256)
    # - Activation functions? Relu is probably fine
    # - L2, L1
    # - Learning rate
    # - Batch size
    # - Epochs (w/ early stopping?)

# 3. NN Programming

# A: 

# - Build NN creation into a function that takes in layers, units, and learning rate
# - Create param grid
# - 


In [28]:
def build_model(num_layers, lr, x_data, last_reg_layer):

    num_units = np.zeros(num_layers, dtype=int)
    num_units[-1] = 1
    for i in range(num_layers-1):
        num_units[i] = 16 * 2**(num_layers-2-i)

    layers = []
    ki = GlorotUniform(seed=2434)
    layers.append(Dense(num_units[0], input_shape=(x_data.shape[1],), activation='relu', kernel_initializer=ki))
    for i in range(1, num_layers-1):
        layers.append(Dense(num_units[i], activation='relu', kernel_initializer=ki))
    layers.append(Dense(1, kernel_initializer=ki, kernel_regularizer=last_reg_layer))


    model = Sequential(layers)

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)

    model.compile( optimizer=optimizer, loss="mse", metrics=["mae"] )

    return model

In [29]:
from tensorflow.keras import regularizers
param_grid = {
    'num_layers': [3, 4, 5],
    'lr': [1e-3, 1e-2, 1e-1],
    'last_reg_layer': [None, regularizers.l1(1e-3), regularizers.l1(1e-2), regularizers.l1(1e-1), regularizers.l2(1e-3), regularizers.l2(1e-2), regularizers.l2(1e-1)]
}

In [30]:
from itertools import product

# Create all combinations
all_combos = list(product(param_grid['num_layers'], 
                          param_grid['lr'], 
                          param_grid['last_reg_layer']))


In [37]:
results = []

for num_layers, lr, last_reg_layer, in all_combos:

    model = build_model(num_layers, lr, XtrainNN_norm_df, last_reg_layer)

    history = model.fit(
        x = XtrainNN_norm_df,
        y = ytrainNN_norm_df,
        epochs=10,
        validation_data = (XvalidNN_norm_df, yvalidNN_norm_df),
        verbose=0
    )

    y_hat_valid = model.predict(XvalidNN_norm_df)
    perf_valid = r2_score(yvalidNN_norm_df, y_hat_valid)

    y_hat_train = model.predict(XtrainNN_norm_df)
    perf_train = r2_score(ytrainNN_norm_df, y_hat_train)

    results.append({
    'num_layers': num_layers,
    'lr': lr,
    'last_reg_layer': last_reg_layer,
    'r2_train': perf_train,
    'r2_valid': perf_valid,
    'model': model
    })

results_df = pd.DataFrame(results)

print(results_df.head())



/opt/miniconda3/envs/ME276/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step


/opt/miniconda3/envs/ME276/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step


/opt/miniconda3/envs/ME276/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step


/opt/miniconda3/envs/ME276/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step


/opt/miniconda3/envs/ME276/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step
   num_layers     lr                                     last_reg_layer  \
0           3  0.001                                               None   
1           3  0.001  <keras.src.regularizers.regularizers.L1 object...   
2           3  0.001  <keras.src.regularizers.regularizers.L1 object...   
3           3  0.001  <keras.src.regularizers.regularizers.L1 object...   
4           3  0.001  <keras.src.regularizers.regularizers.L2 object...   

   r2_train  r2_valid                                         model  
0  0.103747 -0.056638  <Sequential name=sequential_317, built=True>  
1  0.103334 -0.068340  <Sequential name=sequential_318, built=True>  
2  0.102927 -0.061152  <Sequential name=sequential_319, built=True>  
3  0.090503 -0.052448  <Sequential name=sequential_320, built=True>  
4  0.105584 -0.068817  <Sequential name=sequential_321, built=True>  


---

<h1><center>Random Forest

---

In [38]:
results_df.head()

,num_layers,lr,last_reg_layer,r2_train,r2_valid,model
0,3,0.001,None,0.103747,-0.056638,"<Sequential name=sequential_317, built=True>"
1,3,0.001,<keras.src.regularizers.regularizers.L1 object...,0.103334,-0.068340,"<Sequential name=sequential_318, built=True>"
2,3,0.001,<keras.src.regularizers.regularizers.L1 object...,0.102927,-0.061152,"<Sequential name=sequential_319, built=True>"
3,3,0.001,<keras.src.regularizers.regularizers.L1 object...,0.090503,-0.052448,"<Sequential name=sequential_320, built=True>"
4,3,0.001,<keras.src.regularizers.regularizers.L2 object...,0.105584,-0.068817,"<Sequential name=sequential_321, built=True>"


---

<h1><center>Extra Workspace

---